# 👋🌍 Hello, world: Submit a Cirq job to Quantinuum

In this notebook, we'll review the basics of Azure Quantum by submitting a simple *job*, or quantum program, to [Quantinuum](https://www.quantinuum.com/). We will use [Cirq](https://quantumai.google/cirq) to express the quantum job.

## Submit a simple job to Quantinuum using Azure Quantum
Azure Quantum provides several ways to express quantum programs. In this example, we are using Cirq, but note that Q# and Qiskit are also supported. All code in this example will be written in Python.

Let's begin. When you see a code block, hover over it and click the triangle play-button to execute it. To avoid any compilation issues, this should be done in order from top to bottom.

### 1. Connect to the Azure Quantum workspace

To connect to the Azure Quantum service, construct an instance of the `AzureQuantumService`. Note that it's imported from `azure.quantum.cirq`.

In [ ]:
from azure.quantum.cirq import AzureQuantumService
service = AzureQuantumService (
    resource_id = "",
    location = "",
)

Let's see what providers and targets are enabled in this workspace with the following command:


In [ ]:
print("This workspace's targets:")
for target in service.targets():
    print("-", target.name)

### ❕ Do you see `quantinuum.sim.h1-1sc` in your list of targets? If so, you're ready to keep going.

Don't see it? You may need to add Quantinuum to your workspace to run this sample. Navigate to the **Providers** page in the portal and click **+Add** to add the Quantinuum provider. Don't worry, there's a free credits plan available.

## Quantinuum: The quantum provider
Azure Quantum partners with third-party companies to deliver solutions to quantum jobs. These company offerings are called *providers*. Each provider can offer multiple *targets* with different capabilities. See the table below for Quantinuum's H1-1 device targets.

| Target name | Target ID | Number of qubits | Description|
| --- | ---| ---|---|
H1-1 Syntax Checker | `quantinuum.sim.h1-1sc` | 20 | Quantinuum's H1-1 Syntax Checker. This will return all zeros in place of actual or simulated results. Use this to validate quantum programs against the H1-1 compiler before submitting to hardware or emulators on Quantinuum's platform. Free of cost. |
H1-2 Syntax Checker | `quantinuum.sim.h1-2sc` | 20 | Quantinuum's H1-2 Syntax Checker. This will return all zeros in place of actual or simulated results. Use this to validate quantum programs against the H1-2 compiler before submitting to hardware or emulators on Quantinuum's platform. Free of cost.  |
H2-1 Syntax Checker | `quantinuum.sim.h2-1sc` | 32 | Quantinuum's H2-1 Syntax Checker. This will return all zeros in place of actual or simulated results. Use this to validate quantum programs against the H2-1 compiler before submitting to hardware or emulators on Quantinuum's platform. Free of cost.  |
H1-1 Emulator | `quantinuum.sim.h1-1e` | 20 | Quantinuum's H1-1 Emulator. Uses a realistic physical model and noise model of H1-1. |
H1-2 Emulator | `quantinuum.sim.h1-2e` | 20 | Quantinuum's H1-2 Emulator. Uses a realistic physical model and noise model of H1-2. |
H2-1 Emulator | `quantinuum.sim.h2-1e` | 32 | Quantinuum's H2-1 Emulator. Uses a realistic physical model and noise model of H2-1. |
H1-1 | `quantinuum.qpu.h1-1` | 20 | Quantinuum's H1-1 trapped ion device. |
H1-2 | `quantinuum.qpu.h1-2` | 20 | Quantinuum's H1-2 trapped ion device. |
H2-1 | `quantinuum.qpu.h2-1` | 32 | Quantinuum's H2-1 trapped ion device. |

For this example, we will use `quantinuum.sim.h1-1sc` to avoid any costs or credit usage. If you wish to emulate or run the actual circuit, you may replace all instances of `quantinuum.sim.h1-1sc` in subsequent code cells with one of the other values in the table above, but please note any costs incurred. To learn more about Quantinuum's targets, check out our [documentation](https://aka.ms/AQ/Quantinuum/Documentation).

### 2. Build the quantum program

Let's create a simple Cirq circuit to run.

In [ ]:
import cirq

q0 = cirq.LineQubit(0)
circuit = cirq.Circuit(
    cirq.H(q0),                # Apply an H-gate to q0
    cirq.measure(q0, key="0")  # Measure q0
)
circuit

The circuit you built is a simple quantum random bit generator. With Quantinuum's Syntax Checker, we will be able to confirm that the circuit is able to be run on their H1 emulator and hardware.

### 3. Submit the quantum program to Quantinuum

In [ ]:
# Using the Quantinuum Syntax Checker target, call "run" to submit the job. We'll
# use 100 repetitions (simulated runs).
job = service.targets("quantinuum.sim.h1-1sc").submit(circuit, name="hello world-cirq-quantinuum", repetitions=100)

# Print the job ID.
print("Job id:", job.job_id())

The job ID can be used to retrieve the results later using the `get_job` function or by viewing it under the **Job management** section of the portal.

### 4. Obtain the job results
To get the job's results, you can await completion and plot the output, as in the next code cell. This may take a few seconds.

In [ ]:
# Await job results.
result = job.results()

from matplotlib import pyplot

# Quantinuum's emulators and quantum computers do not return probabilities, they return a dictionary of measurements.
# (Note that since we targeted Syntax Checker, we expect to see that all measurements result in a zero.)
# In order to visualize the result, we can use a histogram. Since the circuit contains only a single
# measurement, we can expect that all values will be tied to the key "m_0".
pyplot.hist(result['m_0'], bins=[0,1,2], width=0.5)
pyplot.title("Result")
pyplot.xlabel("Measurement")
pyplot.ylabel("Repetitions")

pyplot.show()

**See the histogram above? Congratulations, you've submitted a job with Azure Quantum! 👏**


### 5. Estimate costs

To estimate the costs of running this program on a simulator or hardware, you can use the `service.estimate_cost` method.

In [ ]:
cost = service.estimate_cost(circuit, repetitions=100, target="quantinuum.qpu.h1-1")
print(f"Estimated cost: {cost.estimated_total} {cost.currency_code}")

### 6. Next steps
Next, you can try running a program on Quantinuum's emulation and hardware targets. Just replace `quantinuum.sim.h1-1sc` with `quantinuum.sim.h1-1e` or `quantinuum.qpu.h1-1`. Or try another sample by navigating back to the sample gallery. The same "hello world" sample can be run with different quantum providers by choosing another option in the gallery card drop-down menu. Don't worry - your work here is automatically saved.

To learn more about submitting Cirq circuits to Azure Quantum, review [this documentation](https://docs.microsoft.com/azure/quantum/quickstart-microsoft-cirq?pivots=platform-quantinuum).

To learn more about job pricing, review the [Azure Quantum documentation on job costs](https://docs.microsoft.com/azure/quantum/azure-quantum-job-costs).